In [4]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key =openai_api_key

In [ ]:
# CSV 파일 불러오기 (첫 번째 행 제외)
file_path='../5word.csv'
df_ex = pd.read_csv(file_path)

In [ ]:
df_ex.head()

,KO,ENG,JPN
0,경북대학교,Kyungpook National University,慶北大学校
1,경북대학교 대구캠퍼스박물관,Kyungpook National University Daegu Campus Museum,慶北大学校大邱キャンパス博物館
2,경북대학교치과대학,Kyungpook National University Dental College,慶北大学歯科大学
3,계명대학교 성서캠퍼스행소박물관,Keimyung University Sungseo Campus Action Museum,啓明大学城西キャンパス行所博物館
4,계명문화대학교 평생교육원,Keimyung University Lifelong Education Center,啓明文化大学生涯教育院


In [8]:
from langsmith import trace
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
    
def generate_sentences_from_csv(df, openai_api_key):
    
    # Initialize OpenAI model
    llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
    
    results = []
    
    # Define prompt templates
    korean_prompt = PromptTemplate.from_template(
        """
        You are a professional Korean language educator.
        Create a natural Korean sentence that includes the word '{word}'.
        Provide only the sentence without the original sentence, additional examples, symbols, or punctuation explanations.
        """
        )
    english_prompt = PromptTemplate.from_template(
        """
        You are a professional English translator.
        Translate the following Korean sentence into English using the word '{hint}': {sentence}.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        """)
    japanese_prompt = PromptTemplate.from_template(
        """
        You are a professional Japanese translator.
        Translate the following Korean sentence into Japanese using the word '{hint}': {sentence}.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        """)
    
    # Define chains
    korean_chain = LLMChain(llm=llm, prompt=korean_prompt)
    english_chain = LLMChain(llm=llm, prompt=english_prompt)
    japanese_chain = LLMChain(llm=llm, prompt=japanese_prompt)
    
    for index, row in df.iterrows():
        korean_word = row.iloc[0]  # 한국어
        english_hint = row.iloc[1]  # 영어
        japanese_hint = row.iloc[2]  # 일본어
        
        korean_sentence = korean_chain.run(word=korean_word)
        english_translation = english_chain.run(hint=english_hint, sentence=korean_sentence)
        japanese_translation = japanese_chain.run(hint=japanese_hint, sentence=korean_sentence)
        
        results.append([ korean_word, korean_sentence, english_translation, japanese_translation])
    
    result_df = pd.DataFrame(results, columns=[ "Word","Korean Sentence", "English Translation", "Japanese Translation"])
    result_df.to_csv("translated_sentences.csv", index=False, encoding="utf-8-sig")
    
    print("저장됨v")
    

generate_sentences_from_csv(df, openai_api_key)

C:\Users\201\AppData\Local\Temp\ipykernel_16120\1540467423.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
C:\Users\201\AppData\Local\Temp\ipykernel_16120\1540467423.py:35: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  korean_chain = LLMChain(llm=llm, prompt=korean_prompt)
C:\Users\201\AppData\Local\Temp\ipykernel_16120\1540467423.py:44: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  kore

저장됨v


In [2]:
from langchain.chains import SequentialChain

def generate_sentences_from_csv(df, openai_api_key):
    from langchain.chat_models import ChatOpenAI
    from langchain.prompts import PromptTemplate
    import json
    import pandas as pd

    llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
    results = []

    korean_prompt = PromptTemplate.from_template(
        """
        You are a professional Korean language educator.
        Create a natural Korean sentence that includes the word '{word}'.
        Provide only the sentence without the original sentence, additional examples, symbols, or punctuation explanations.
        """
    )

    english_prompt = PromptTemplate.from_template(
        """
        You are a professional English translator.
        Translate the following Korean sentence into English using the word '{hint}': {sentence}.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        """
    )

    # Define the sequential chain with both Korean and English chains
    korean_chain = LLMChain(llm=llm, prompt=korean_prompt)
    english_chain = LLMChain(llm=llm, prompt=english_prompt)
    
    sequential_chain = SequentialChain(
        chains=[korean_chain, english_chain],
        input_variables=["word", "hint", "sentence"],
        output_variables=["korean_sentence", "english_translation"]
    )
    
    for index, row in df.iterrows():
        metadata_json = row["metadata"]
        metadata_dict = json.loads(metadata_json)  # JSON 문자열을 딕셔너리로 변환
        
        korean_word = metadata_dict["KO"]
        english_hint = metadata_dict["ENG"]
        
        # Run the sequential chain
        output = sequential_chain.run(word=korean_word, hint=english_hint, sentence="{korean_sentence}")
        
        korean_sentence = output["korean_sentence"]
        english_translation = output["english_translation"]
        
        instruction_sentence = "Translate the Korean into English. Refer to the glossary, and if it is not in the glossary, translate it. Do not provide explanations."
        language = "ENG"
        
        results.append([instruction_sentence, korean_sentence, english_translation, language, metadata_json])
    
    result_df = pd.DataFrame(results, columns=["instruction", "input", "output", "language", "metadata"])
    result_df.to_excel("transKo_Eng.xlsx", index=False)

    print("저장됨_한영")

# 변환된 엑셀을 불러와서 실행
generate_sentences_from_csv(df, openai_api_key)


NameError: name 'df' is not defined